## *Création d'un model xgboost + Envoi Kaggle*

In [1]:
from pandas import read_csv
from numpy import random
from sklearn.model_selection import train_test_split

### *Import Train full*

In [2]:
train_full = read_csv('files_csv/origin_dataset_modif.csv', index_col=0)

#### *SEED*

In [3]:
SEED = 42
random.seed(SEED)

#### Test Auc en fonction de split différents ( pour avoir le meilleur split )

In [4]:
# from xgboost import XGBClassifier
# from sklearn import metrics

# def split_auc(n):
    
#     SEED = 42
#     random.seed(SEED)
    
#     VAL_SIZE = n
#     target_column = 'SeriousDlqin2yrs'
#     train, val = train_test_split(train_full, test_size=VAL_SIZE)
    
#     path_train = f"test_split/train_sk_{1-n}.csv"
#     path_val = f"test_split/val_sk_{n}.csv"

#     train.to_csv(path_train)
#     val.to_csv(path_val)
    
#     # Load prepared dataset 
    
#     train_prepared = read_csv(path_train, index_col=0)
#     val_prepared = read_csv(path_val, index_col=0)
    
#     # On separe Output des autres valeurs dans le train
#     X_train = train_prepared.drop(target_column, axis=1).values
#     y_train = train_prepared[target_column].values
    
#     # On separe Output des autres valeurs dans le val
#     X_val = val_prepared.drop(target_column, axis=1).values
#     y_val = val_prepared[target_column].values
    
#     # on entraine un modele
#     model = XGBClassifier()
#     model.fit(X_train, y_train)
 
#     # Verification 
#     y_val_proba = model.predict_proba(X_val)
#     y_val_proba = y_val_proba[:,1] # 1 Proba 
#     # AUC 
#     auc = metrics.roc_auc_score(y_val, y_val_proba)
#     return auc

# i = 0.1
# best_auc = 0
# best_split_val = 0
# while i <= 0.9:
#     print(f" Pour train = {1-i} et val =  {i} , auc = {split_auc(i)}")
#     if split_auc(i) > best_auc:
#         best_auc = split_auc(i)
#         best_split_val = i
#     i += 0.1
    
    
    
# print(f"{best_split_val} : {best_auc}")
    
     

#### *Split train full*

In [5]:
VAL_SIZE = 0.3 # Best auc with val_Size = 0.3
train, val = train_test_split(train_full, test_size=VAL_SIZE)

print("Train full shape: " + str(train_full.shape))
print("Train shape: " + str(train.shape))
print("Val shape: " + str(val.shape))

train.to_csv("files_csv/train_sk.csv")
val.to_csv("files_csv/val_sk.csv")

Train full shape: (150000, 18)
Train shape: (105000, 18)
Val shape: (45000, 18)


### *Load prepared dataset*

In [6]:
train_prepared = read_csv('files_csv/train_sk.csv', index_col=0)
%store train_prepared
val_prepared = read_csv('files_csv/val_sk.csv', index_col=0)

Stored 'train_prepared' (DataFrame)


#### *y_train = Seriousdlsin2yrs*

In [7]:
target_column = 'SeriousDlqin2yrs'
y_train = train_prepared[target_column].values
print(y_train)

[0 0 0 ... 0 0 0]


#### Tout le train sans la colonne output

In [8]:
from pprint import pprint
X_train = train_prepared.drop(target_column, axis=1).values
pprint(X_train)

array([[1.86299478e-01, 5.50000000e+01, 0.00000000e+00, ...,
        2.19745064e+03, 0.00000000e+00, 5.00000000e+00],
       [7.60345836e-01, 5.50000000e+01, 1.00000000e+00, ...,
        8.40214984e+03, 1.00000000e+00, 9.00000000e+00],
       [1.01499674e-01, 3.90000000e+01, 0.00000000e+00, ...,
        6.38117503e+03, 0.00000000e+00, 4.00000000e+00],
       ...,
       [6.87882910e-02, 4.90000000e+01, 0.00000000e+00, ...,
        9.14673332e+02, 0.00000000e+00, 6.00000000e+00],
       [2.19630062e-01, 3.60000000e+01, 0.00000000e+00, ...,
        1.46202532e+03, 0.00000000e+00, 2.00000000e+00],
       [2.08050838e-01, 6.20000000e+01, 1.00000000e+00, ...,
        8.94810519e+03, 2.00000000e+00, 4.00000000e+00]])


#### *Pareil pour le val_set*

In [9]:
X_val = val_prepared.drop(target_column, axis=1).values
y_val = val_prepared[target_column].values

### *Prepared train set*

In [10]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
%store model

Stored 'model' (XGBClassifier)


### *Apply model to val set*

In [11]:
y_val_proba = model.predict_proba(X_val)
print(y_val_proba) # 1 Proba

[[0.99076664 0.00923336]
 [0.988792   0.01120799]
 [0.9894754  0.01052456]
 ...
 [0.94266886 0.05733114]
 [0.7797888  0.22021121]
 [0.8398806  0.1601194 ]]


In [12]:
y_val_proba = y_val_proba[:,1]
print(y_val_proba)

[0.00923336 0.01120799 0.01052456 ... 0.05733114 0.22021121 0.1601194 ]


In [13]:
from sklearn import metrics
metrics.roc_auc_score(y_val, y_val_proba)

0.8649011097304256

### *Envoi Kaggle*

#### *Prédiction sur le fichier kaggle et mise en forme*

In [14]:
import pandas as pd

kaggle_prepared = read_csv('files_csv/kaggle_dataset_modif.csv', index_col=0)
X_kag = kaggle_prepared.drop(target_column, axis=1).values
y_kag = kaggle_prepared[target_column].values
y_kag_proba =  model.predict_proba(X_kag)[:,1] # toutes les lignes colonnes 1


Id = [i+1 for i in range(len(X_kag))]
d = {'Id': Id, 'Probability': y_kag_proba}
kaggle_df = pd.DataFrame(data=d)
kaggle_df.to_csv("files_csv/prediction_sk.csv", index=False)

#### *Envoi Kaggle*

In [15]:
import kaggle
submission_file = 'files_csv/prediction_sk.csv'
kaggle.api.competition_submit(submission_file, "BigML ensemble", "GiveMeSomeCredit")

100%|██████████| 1.70M/1.70M [00:05<00:00, 315kB/s] 


Successfully submitted to Give Me Some Credit